In [1]:
import numpy as np
import pandas as pd
import h5py
import json
from itertools import compress
from datetime import datetime
from calfews_src import *
from calfews_src.util import *
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
from bqplot import pyplot as plt
from bqplot import LinearScale, DateScale, Lines, Axis, Figure
from bqplot.interacts import BrushSelector, BrushIntervalSelector, panzoom
import threading
import time
import tornado
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

In [2]:
### flags for command line, diff for windows vs unix
if os.name == 'nt':
    mkdir_os = 'mkdir '
    sep_os = '\\'
else:
    mkdir_os = 'mkdir -p '
    sep_os = '/'
    
### flip slash
def slash_os(s):
    return s.replace('/', sep_os)

#######################################################################
### set up functions
#######################################################################

### function to run simulation using chosen options
def run_sim(progress_w, ioloop):
    global sim_finished, main_cy_obj
    
    ### delete old results, then recreate folder to store new results
    !rm -rf {slash_os(temp_results_sim + filname)}
    !{mkdir_os} {slash_os(temp_results_sim + filname)}    

    ### initialize model
    import main_cy
    main_cy_obj = main_cy.main_cy()   

    ### setup thread to continuously check model progress, end when model finishes
    def update_progress(progress, progress_w=progress_w):
        progress_w.value = progress
    def check_progress_thread():
        ### update progress bar asynchronously
        while not sim_finished.isSet():
            sim_finished.wait(5)
            ioloop.add_callback(update_progress, main_cy_obj.progress)
    
    t_check_progress = threading.Thread(target = check_progress_thread)
    t_check_progress.start()   
    
    ### run model
    main_cy_obj.run_py(temp_results_sim + filname, model_mode, flow_input_type, flow_input_source)
    
    ### move results to permanent storage (if different)
    if temp_results_sim != permanent_results:
        !cp -r {temp_results_sim + filname} {permanent_results}
        print('Copying data to ' + permanent_results + filname)
    
    print('#######################################################')

    print('Finished! Proceed to "Plot data" tab.')
    
    

### function to deal with different label in gui label vs dataset
def get_alternative_label(output_type, bin_is_attribute, new_column, old_column, old_label):
    if output_type in ['District'] and bin_is_attribute == 1 and old_column == 'Object Attributes':
        ### check if label contains Contract/District/Waterbank
        try:
            name, old_att = old_label.split('_', 1)
            if name in results_keys['Contract'][0]['Object List Model'].values.tolist():
                new_name = get_alternative_label('Contract', 0, 'Object Name', 'Object List Model', name)
                new_att = get_alternative_label('District', 1, 'Label', 'Object Attributes', '*contract_' + old_att)
                return new_name + ': ' + new_att
            elif name in results_keys['District'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('District', 0, 'Object Name', 'Object Key', name)
                new_att = get_alternative_label('District', 1, 'Label', 'Object Attributes', '*district_' + old_att)
                return new_name + ': ' + new_att
            elif name in results_keys['Waterbank'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('Waterbank', 0, 'Object Name', 'Object Key', name)
                new_att = get_alternative_label('District', 1, 'Label', 'Object Attributes', '*waterbank_' + old_att)
                return new_name + ': ' + new_att            
            else:
                pass
        except:
#           ### check if key is Private/District with bank account at this District
            if old_label in results_keys['Private'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('Private', 0, 'Object Name', 'Object Key', old_label)
                new_att = get_alternative_label('District', 1, 'Label', 'Object Attributes', '*bank_member_key')
                return new_name + ': ' + new_att     
            if old_label in results_keys['District'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('District', 0, 'Object Name', 'Object Key', old_label)
                new_att = get_alternative_label('District', 1, 'Label', 'Object Attributes', '*bank_member_key')
                return new_name + ': ' + new_att                 
            else:
                pass
    elif output_type in ['District'] and bin_is_attribute == 1 and old_column == 'Label':
        ### check if label containts Contract/District/Waterbank
        try:
            name, old_att = old_label.split(': ', 1)
            if name in results_keys['Contract'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('Contract', 0, 'Object List Model', 'Object Name', name)
                new_att = get_alternative_label('District', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return get_alternative_label('Contract', 0, 'Object Key', 'Object List Model', new_name)
                else:
                    return new_att.replace('*contract', new_name)
            elif name in results_keys['District'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('District', 0, 'Object Key', 'Object Name', name)
                new_att = get_alternative_label('District', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return new_name
                else:
                    return new_att.replace('*district', new_name)
            elif name in results_keys['Waterbank'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('Waterbank', 0, 'Object Key', 'Object Name', name)
                new_att = get_alternative_label('District', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return new_name
                else:
                    return new_att.replace('*waterbank', new_name)
                
            elif name in results_keys['Private'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('Private', 0, 'Object Key', 'Object Name', name)
                new_att = get_alternative_label('District', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return new_name
                else:
                    return new_att.replace('*private', new_name)  
            else:
                pass
        except:
            pass
        
    elif output_type in ['Private'] and bin_is_attribute == 1 and old_column == 'Object Attributes':
        ### check if label contains Contract/District/Waterbank
        try:
            name1, old_att1 = old_label.split('_', 1)
            if name1 in results_keys['District'][0]['Object Key'].values.tolist():
                new_name1 = get_alternative_label('District', 0, 'Object Name', 'Object Key', name1)
                ### check if attribute contains Contract/District/Waterbank
                try:
                    name2, old_att2 = old_att1.split('_', 1)
                    if name2 in results_keys['Contract'][0]['Object List Model'].values.tolist():
                        new_name2 = get_alternative_label('Contract', 0, 'Object Name', 'Object List Model', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Label', 'Object Attributes', '*location_*contract_' + old_att2)
                        return new_name1 + ': ' + new_name2 + ': ' + new_att2
                    elif name2 in results_keys['District'][0]['Object Key'].values.tolist():
                        new_name2 = get_alternative_label('District', 0, 'Object Name', 'Object Key', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Label', 'Object Attributes', '*location_*district_' + old_att2)
                        return new_name1 + ': ' + new_name2 + ': ' + new_att2
                    elif name2 in results_keys['Waterbank'][0]['Object Key'].values.tolist():
                        new_name2 = get_alternative_label('Waterbank', 0, 'Object Name', 'Object Key', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Label', 'Object Attributes', '*location_*waterbank_' + old_att2)
                        return new_name1 + ': ' + new_name2 + ': ' + new_att2
                    else:
                        pass
                except:
                    pass
        except:
            pass
        
    elif output_type in ['Private'] and bin_is_attribute == 1 and old_column == 'Label':
        ### first should be key for District associated with Private water account
        try:
            name1, old_att1 = old_label.split(': ', 1)
            if name1 in results_keys['District'][0]['Object Name'].values.tolist():
                new_name1 = get_alternative_label('District', 0, 'Object Key', 'Object Name', name1)

                ## check if attribute contains Contract/District/Waterbank
                try:            
                    name2, old_att2 = old_att1.split(': ', 1)

                    if name2 in results_keys['Contract'][0]['Object Name'].values.tolist():
                        new_name2 = get_alternative_label('Contract', 0, 'Object List Model', 'Object Name', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Object Attributes', 'Label', old_att2)
                        return new_att2.replace('*location_*contract', new_name1 + '_' + new_name2)
                    elif name2 in results_keys['District'][0]['Object Name'].values.tolist():
                        new_name2 = get_alternative_label('District', 0, 'Object Key', 'Object Name', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Object Attributes', 'Label', old_att2)
                        return new_att2.replace('*location_*district', new_name1 + '_' + new_name2)
                    elif name2 in results_keys['Waterbank'][0]['Object Name'].values.tolist():
                        new_name2 = get_alternative_label('Waterbank', 0, 'Object Key', 'Object Name', name2)
                        new_att2 = get_alternative_label('Private', 1, 'Object Attributes', 'Label', old_att2)
                        return new_att2.replace('*location_*waterbank', new_name1 + '_' + new_name2)
                    else:
                        pass
                except:
                    pass
        except:
            pass    
        
    elif output_type in ['Waterbank'] and bin_is_attribute == 1 and old_column == 'Object Attributes':
        ### check if label contains Contract/District/Waterbank
        try:
#           ### check if key is Private/District with bank account at this District 
            if old_label in results_keys['District'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('District', 0, 'Object Name', 'Object Key', old_label)
                new_att = get_alternative_label('Waterbank', 1, 'Label', 'Object Attributes', '*bank_member_key')
                return new_name + ': ' + new_att                 
            elif old_label in results_keys['Private'][0]['Object Key'].values.tolist():
                new_name = get_alternative_label('Private', 0, 'Object Name', 'Object Key', old_label)
                new_att = get_alternative_label('Waterbank', 1, 'Label', 'Object Attributes', '*bank_member_key')
                return new_name + ': ' + new_att                
            else:
                pass
        except:
            pass
    elif output_type in ['Waterbank'] and bin_is_attribute == 1 and old_column == 'Label':
        ### check if label containts Contract/District/Waterbank
        try:
            name, old_att = old_label.split(': ', 1)
            if name in results_keys['District'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('District', 0, 'Object Key', 'Object Name', name)
                new_att = get_alternative_label('Waterbank', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return new_name
                else:
                    pass                
            elif name in results_keys['Private'][0]['Object Name'].values.tolist():
                new_name = get_alternative_label('Private', 0, 'Object Key', 'Object Name', name)
                new_att = get_alternative_label('Waterbank', 1, 'Object Attributes', 'Label', old_att)
                if new_att == '*bank_member_key':
                    return new_name
                else:
                    pass
            else:
                pass
        except:
            pass
        
    
    labels_df = results_keys[output_type][bin_is_attribute]
    try:
        new_label = labels_df[new_column].loc[labels_df[old_column] == old_label].values[0]
        return new_label
    except:
        return None



            
            
            
            
            
def plot_data_initial():
    global fig, fig_zoom
    xdata = []
    ydata = []
    labels = []
    xsc = DateScale()
    ysc = LinearScale()
    k1 = output_type_w.value
    for output in output_split2_w.value:
        if output_order_w.value == output_orders[k1][0]:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output_split1_w.value)
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output)
        else:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output)
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output_split1_w.value)        
        key = obj_key + '_' + att_key
        xdata.append(datDaily.index)
        ydata.append(datDaily[key])
        labels.append(output)
    lines = Lines(x=xdata, y=ydata, scales={'x':xsc, 'y':ysc}, labels=labels, display_legend=True)
    axx = Axis(scale=xsc, label='Date')
    title = output_split1_w.value
    axy = Axis(scale=ysc, orientation='vertical')    

    ### set up first plot with brushing capability
    brush_selector_w = BrushIntervalSelector(scale=xsc, marks=[lines])
    fig = Figure(marks=[lines], axes=[axx, axy], interaction=brush_selector_w, title=title, legend_location=leg_loc)
    fig.legend_style = {'stroke-width': 0}

#     ### set up second plot, which will be zoomed to box from first plot
    xsc_zoom = DateScale()
    ysc_zoom = LinearScale()
    lines_zoom = Lines(x=xdata, y=ydata, scales={'x':xsc_zoom, 'y':ysc_zoom}, labels=labels, display_legend=True)
    axx_zoom = Axis(scale=xsc_zoom, label='Date')
    axy_zoom = Axis(scale=ysc_zoom, orientation='vertical')    
    fig_zoom = Figure(marks=[lines_zoom], axes=[axx_zoom, axy_zoom], title=title, legend_location=leg_loc)
    fig_zoom.legend_style = {'stroke-width': 0}

    def brush_bounds_update(b):
        if brush_selector_w.selected is not None:
            xsc_zoom.min, xsc_zoom.max = brush_selector_w.selected
        else:
            xsc_zoom.min, xsc_zoom.max = np.min(fig_zoom.marks[0].x), np.max(fig_zoom.marks[0].x)
        fig_zoom.axes[0].scale = xsc_zoom

    brush_selector_w.observe(brush_bounds_update, names=['brushing'])
    
    plot_html3_w = widgets.HTML(value = '<b>Figure 1: Full time series (click and drag to select zoom region for Figure 2)</b><br />\n')
    plot_html4_w = widgets.HTML(value = '<b>Figure 2: Zoomed in time series</b><br />\n')

    display(widgets.VBox([plot_html3_w, fig, plot_html4_w, fig_zoom]))

    

    
def plot_data_output_type(output_type):
    global output_split2_w
    output_split2_w.value = [output_split2_w.options[0]]
    
    


def plot_data_output_split1(output_split1):
    global fig, fig_zoom
    xdata = []
    ydata = []
    labels = []
    k1 = output_type_w.value
    for output in output_split2_w.value:
        if output_order_w.value == output_orders[k1][0]:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output_split1['new'])
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output)
        else:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output)
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output_split1['new'])         
        key = obj_key + '_' + att_key
        xdata.append(datDaily.index)
        ydata.append(datDaily[key])
        labels.append(output)
    fig.marks[0].x = xdata
    fig.marks[0].y = ydata
    fig.marks[0].labels = labels
    fig_zoom.marks[0].x = xdata
    fig_zoom.marks[0].y = ydata
    fig_zoom.marks[0].labels = labels
    title = output_split1['new']
    fig.title = title
    fig_zoom.title = title


     
def plot_data_output_split2(output_split2):
    global fig, fig_zoom
    xdata = []
    ydata = []
    labels = []
    k1 = output_type_w.value
    for output in output_split2['new']:
        if output_order_w.value == output_orders[k1][0]:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output_split1_w.value)
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output)
        else:
            obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output)
            att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output_split1_w.value)  
        key = obj_key + '_' + att_key
        xdata.append(datDaily.index)
        ydata.append(datDaily[key])
        labels.append(output)
    fig.marks[0].x = xdata
    fig.marks[0].y = ydata
    fig.marks[0].labels = labels
    fig_zoom.marks[0].x = xdata
    fig_zoom.marks[0].y = ydata
    fig_zoom.marks[0].labels = labels

    
    
    
        
def update_tab_display(widget):
#     get the correct Output widget based on the index of the chosen tab
    tab_idx = widget['new']
    output_widget = dynamic_outputs[tab_idx]
    with output_widget:
        clear_output()
        tab_dispatcher(tab_idx)
        
        
        
def tab_intro():
    clear_output()
    intro_html = '<b>Graphical User Interface for the California Food-Energy-Water System (CALFEWS) simulation model</b><br />\n'
    intro_html += '<p>This tool provides a user interface for the <a href="https://github.com/hbz5000/CALFEWS">California Food-Energy-Water System (CALFEWS)</a>, an open-sourced, Python-based model for simulating the integrated, multi-sector dynamics of water supply in the Central Valley of California. CALFEWS captures system dynamics across multiple scales, from coordinated management of inter-basin water supply projects at the state and regional scale, to agent-based representation of conjunctive surface water and groundwater supplies at the scale of irrigation and water storage districts. This user interface can be used to simulate operations under different hydrologic conditions and visualize the results using the interactive plotting features.</p><br />\n'
    intro_html += '<p>More information on the CALFEWS model, and comparison of model output to historical data, can be found in the following manuscript:</p><br />\n'
    intro_html +=  '<p>Zeff, H.B., Hamilton, A.L., Malek, K., Herman, J.D., Cohen, J.S., Medellin-Azuara, J., Reed, P.M., and G.W. Characklis. (2020). "California\'s Food-Energy-Water System: An Open Source Simulation Model of Adaptive Surface and Groundwater Management in the Central Valley". (In review, <a href="https://engrxiv.org/sqr7e/")>preprint available here</a>.)</p><br />\n'
    intro_html_w = widgets.HTML(value = intro_html)
    display(intro_html_w)
    


def tab_simulation():
    global model_mode, flow_input_type, flow_input_source, filname_w, filname, sim_finished
    ### This tab will allow user to run a new dataset
    clear_output()
    
    ### add instructions as html
    sim_html1_w = widgets.HTML(value = '<b>Select inflow scenario</b><br />\n')
#     display(sim_html1_w)
    
    
    ### get input file scenarios from key file
    input_key = pd.read_excel('gui_key.xlsx', sheet_name='input_data', engine="openpyxl")
    input_file_options = {input_key['Inflow scenario'][i]: input_key['model_mode'][i] + ':' + input_key['flow_input_type'][i] + ':' + input_key['flow_input_source'][i] for i in range(input_key.shape[0])}
    
    ### select widget for input scenario
    input_select_w = widgets.Select(options = list(input_file_options.keys()))
    input_select_w.value = input_select_w.options[0]


    ### table explaining where data came from
    def make_sim_html_table(headers, values):
        yield '<head><style>table, th, td {border: 1px solid black;}table.center {margin-left: auto;margin-right: auto;}th, td {padding: 6px;}</style></head><body><table>'
        yield '<tr><th>'
        yield '</th><th>'.join(headers)
        yield '</th></tr>'
        for sublist in values:
            yield '<tr><td>'
            yield '</td><td>'.join(sublist)
            yield '</td></tr>'
        yield '</table></body>'
    
    sim_table = [input_key.iloc[i, :3].values.tolist() for i in range(input_key.shape[0])]
    sim_table = [[str(item) for item in l] for l in sim_table]
    
    sim_html_table = ''.join(make_sim_html_table(input_key.columns[:3], sim_table))
    table_layout = widgets.Layout(max_height='308px', overflow_y='auto')#width='70%', 
    sim_html_table_w = widgets.HTML(value = sim_html_table, layout=table_layout)


    
    ### choose filename for saving data
    sim_html2_w = widgets.HTML(value = '<b>Folder name to save results (no spaces)</b><br />\n')
    filname = input_select_w.value.replace(' ', '_')
    model_mode, flow_input_type, flow_input_source = input_file_options[input_select_w.value].split(':')
    filname_w = widgets.Text(value = filname)
    
    ### monitor folder name and replace spaces with '_'
    def update_filname(fil):
        global filname
        filname = fil['new'].replace(' ', '_')
    filname_w.observe(update_filname, 'value')
    
    ### update default folder name when new scenario chosen
    def input_update(input_select):
        global model_mode, flow_input_type, flow_input_source, filname_w
        filname_w.value = input_select['new'].replace(' ', '_')
        model_mode, flow_input_type, flow_input_source = input_file_options[input_select['new']].split(':')
    input_select_w.observe(input_update, 'value')
    
    ### setup progress bar for simulation
    progress_w = widgets.FloatProgress(value=0, min=0, max=1)
    sim_finished = threading.Event()

    def run_sim_thread(*args, **kwargs):
        t_run_sim = threading.Thread(target=run_sim, args=(progress_w, tornado.ioloop.IOLoop.instance()))
        t_run_sim.start()


    ### This button will start simulation and make plotting button available afterwards
    sim_html3_w = widgets.HTML(value = '<b>Click to begin</b><br />\n')
    button = widgets.Button(description='Start')
    button.on_click(run_sim_thread)
#     display(button)

    ### print progress bar
    sim_html4_w = widgets.HTML(value = '<b>Progress</b><br />\n')

#     print('Progress:')
#     display(progress_w)    
    
    
    ### stack widgets for display
    sim_left_display_w = widgets.VBox([sim_html1_w, input_select_w, sim_html2_w, filname_w, sim_html3_w, button, sim_html4_w, progress_w])
    sim_left_display_w.layout.min_width = '310px'
    sim_display_w = widgets.HBox([sim_left_display_w, sim_html_table_w])
    sim_display_w.layout.grid_gap = '12px'
    display(sim_display_w)
      
    
def tab_plotting_dynamic():
    clear_output()
    with static_outputs[2]:
        global data_w, output_plot_w
        ### This tab will allow user to visualize their data
        clear_output()

        ### add instructions as html
        plot_html = '<b>Select dataset</b><br />\n'
        plot_html_w = widgets.HTML(value = plot_html)
        display(plot_html_w)

        folders = !ls {permanent_results}
        try:
            folders.remove('.ipynb_checkpoints')
        except:
            pass
        data_w = widgets.Select(options=folders, value=None)
        display(data_w)
        
#     time.sleep(0.1)
#     data_w.value = None
    data_w.observe(get_data, 'value')



def tab_dispatcher(tab_idx):
    if tab_idx == 0:
        pass
    
    elif tab_idx == 1:
        tab_simulation()
    
    elif tab_idx == 2:
        tab_plotting_dynamic()
        
        
        
        
### function to get all possible output splits for plotting
def get_output_splits():
    global results_keys, output_objects, output_split1s, output_split2s, colsplit1, colsplit2
    
    ### get results attribute keys
    results_keys = {}
    for k1 in ['Reservoir','Delta','Contract','District','Private','Waterbank']:
        results_keys[k1] = {}
        ### get objects within class (e.g. shasta for Reservoir)
        objects = pd.read_excel('gui_key.xlsx', sheet_name=k1, engine="openpyxl")
        ### strip whitespace
        for c in objects.columns:
            try:
                objects[c] = [s.strip() for s in objects[c]]
            except:
                pass
        results_keys[k1][0] = objects
        ### get attributes within class (e.g. storage for Reservoir)
        attributes = pd.read_excel('gui_key.xlsx', sheet_name= k1 + '_attributes', engine="openpyxl")
        ### only include attributes of interest
        attributes = attributes.loc[attributes['Include'] == 1, :]
        ### strip whitespace
        for c in objects.columns:
            try:
                objects[c] = [s.strip() for s in objects[c]]
            except:
                pass
        ### add units to label
        attributes['Label'] = attributes['Label'] + ' (' + attributes['Attribute Unit'] + ')'
        results_keys[k1][1] = attributes
        
    output_objects = {}
    for k1 in output_types:
        output_objects[k1] = results_keys[k1][0]['Object Name'].values.tolist()

    ### get options for dropdown 2 based on choice of dropdown 1
    cols = datDaily.columns
    colsplit = [c.split('_', 1) for c in cols]
    colsplit1 = [c[0] for c in colsplit]
    colsplit2 = [c[1] for c in colsplit]
   
        
    output_split1s = {}
    for k1 in output_types:
        output_split1s[k1] = {}
        for output_order in output_orders[k1]:
            output_split1s[k1][output_order] = {}
            if output_order == output_orders[k1][0]:
                l = []
                for k2 in output_objects[k1]:
                    obj_short = get_alternative_label(k1, 0, 'Object List', 'Object Name', k2) 
                    ind = [c == obj_short for c in colsplit1]
                    if sum(ind) > 0:
                        l.append(k2)    
                    output_split1s[k1][output_order] = l
            else:
                l = []
                for k2 in output_objects[k1]:     
                    obj_short = get_alternative_label(k1, 0, 'Object List', 'Object Name', k2) 
                    ind = [c == obj_short for c in colsplit1]
                    if sum(ind) > 0:
                        ltemp = [get_alternative_label(k1, 1, 'Label', 'Object Attributes', ival) for i,ival in enumerate(colsplit2) if ind[i]]
                        l.append([item for item in ltemp if item])               
                l = [item for sublist in l for item in sublist]
                lunique = []
                for item in l:
                    if item not in lunique:
                        lunique.append(item)
                output_split1s[k1][output_order] = lunique

    ### get options for dropdown 3 based on choice of dropdown 1&2          
    output_split2s = {}
    for k1 in output_types:
        output_split2s[k1] = {}
        for output_order in output_orders[k1]:
            output_split2s[k1][output_order] = {}
            for k2 in output_split1s[k1][output_order]:
                if output_order == output_orders[k1][0]:
                    obj_short = get_alternative_label(k1, 0, 'Object List', 'Object Name', k2) 
                    ind = [c == obj_short for c in colsplit1]
                    if sum(ind) > 0:
                        ltemp = [get_alternative_label(k1, 1, 'Label', 'Object Attributes', ival) for i,ival in enumerate(colsplit2) if ind[i]]
                        output_split2s[k1][output_order][k2] = [item for item in ltemp if item]
                        
                else:
                    att_short = get_alternative_label(k1, 1, 'Object Attributes', 'Label', k2) 
                    ind = [c == att_short for c in colsplit2]
                    if sum(ind) > 0:
                        ltemp = [get_alternative_label(k1, 0, 'Object Name', 'Object List', ival) for i,ival in enumerate(colsplit1) if ind[i]]
                        output_split2s[k1][output_order][k2] = [item for item in ltemp if item]



### function to retrieve dataset
def get_data(data):  
    global datDaily, output_type_w, output_split2_w, output_split1_w, output_order_w, output_orders, results_keys, output_types

    with dynamic_outputs[2]:
        clear_output()
        
        ### load data once sim finished
        # results hdf5 file location from CALFEWS simulations
        output_folder_sim = permanent_results + data['new'] + '/'
        got_data = False
        
#         try:
        ### copy results from permanent storage to app file tree (if different)
        if permanent_results != temp_results_plt:
            !cp -r {output_folder_sim} {temp_results_plt}


        print('Loading data from ' + output_folder_sim)
        datDaily = get_results_sensitivity_number_outside_model(temp_results_plt + data['new'] + '/results.hdf5', '')
        got_data = True
            
#         except:
#             print('Enter folder name with valid results')            

        if got_data:
            ### create interactive dropdown selectors for which data to plot
            output_types = ['Reservoir', 'Delta', 'Contract', 'District', 'Private', 'Waterbank']
            output_orders = {'Reservoir': ['Choose reservoir, compare attributes', 'Choose attribute, compare reservoirs'], 
                             'Delta': ['Compare delta attributes'], 
                             'Contract': ['Choose contract, compare attributes', 'Choose attribute, compare contracts'], 
                             'District': ['Choose district, compare attributes', 'Choose attribute, compare districts'], 
                             'Private': ['Choose private, compare attributes', 'Choose attribute, compare privates'], 
                             'Waterbank': ['Choose waterbank, compare attributes', 'Choose attribute, compare waterbank']}
            
            ### get possible plotting combinations for dropdown menu
            get_output_splits()
            

            ### interactive dropdown menu for data selection
            plot_html1_w = widgets.HTML(value = '<b>Select subset of data for figures</b><br />\n')
            
            output_type_w = widgets.Dropdown(options = output_types)
            output_order_w = widgets.Dropdown(options = output_orders[output_type_w.value])
            output_split1_w = widgets.Dropdown(options = output_split1s[output_type_w.value][output_order_w.value])
            output_split2_w = widgets.SelectMultiple(options = output_split2s[output_type_w.value][output_order_w.value][output_split1_w.value], value=[output_split2s[output_type_w.value][output_order_w.value][output_split1_w.value][0]])

            
            
            
            ### create buttons for data download
            plot_html2_w = widgets.HTML(value = '<b>Download full dataset or subset of data</b><br />\n')

            ### button to save full data as csv and trigger download link
            def create_download_full(b):
                ### get alternative label for each column in datDaily
                labels = []
                include = []
                for c in datDaily.columns:
                    name, att = c.split('_', 1)
                    for obj_type in ['Reservoir','Delta','Contract','District','Private','Waterbank']:
                        if name in results_keys[obj_type][0]['Object List'].values:
                            try:
                                new_name = get_alternative_label(obj_type, 0, 'Object Name', 'Object List', name)
                                new_att = get_alternative_label(obj_type, 1, 'Label', 'Object Attributes', att) 
                                labels.append(new_name + ': ' + new_att)
                                include.append(True)                                
                                break
                            except:
                                include.append(False)

                datTemp = datDaily.loc[:, include]
                datTemp.columns = labels
                        
                ### make temp folder to hold results (has to be inside temporary instance of tool, FileLink can't access home directory since it is "upstream" of jupyter home dir)
                !{mkdir_os} temp
                datTemp.to_csv('temp/results_full.csv')
                full_download = FileLink('temp/results_full.csv', result_html_prefix="Click for full dataset: ")
                display(full_download)

            ### This button will start simulation and make plotting button available afterwards
            button_download_full = widgets.Button(description='Full dataset')
            button_download_full.on_click(create_download_full)

            ### button to save full data as csv and trigger download link
            def create_download_subset(b):
                keys = []
                labels = []
                k1 = output_type_w.value
                for output in output_split2_w.value:
                    if output_order_w.value == output_orders[k1][0]:
                        obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output_split1_w.value)
                        att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output)
                    else:
                        obj_key = get_alternative_label(k1, 0, 'Object List', 'Object Name', output)
                        att_key = get_alternative_label(k1, 1, 'Object Attributes', 'Label', output_split1_w.value)        
                    keys.append(obj_key + '_' + att_key)
                    labels.append(output_split1_w.value + ': ' + output)
                datDaily_subset = datDaily.loc[:, keys]
                datDaily_subset.columns = labels
                ### make temp folder to hold results (has to be inside temporary instance of tool, FileLink can't access home directory since it is "upstream" of jupyter home dir)
                !{mkdir_os} temp
                datDaily_subset.to_csv('temp/results_subset.csv')
                subset_download = FileLink('temp/results_subset.csv', result_html_prefix="Click for subsetted dataset: ")
                display(subset_download)

            ### This button will start simulation and make plotting button available afterwards
            button_download_subset = widgets.Button(description='Subsetted dataset')
            button_download_subset.on_click(create_download_subset)
            
            buttons_download_w = widgets.HBox([button_download_full, button_download_subset])



            ### box for displaying data selection
            plot_top_display_w = widgets.VBox([plot_html1_w, output_type_w, output_order_w, output_split1_w, output_split2_w])
            display(plot_top_display_w)


            def update_output_order_w(output_type):
                global output_order_w          
                output_order_w.options = output_orders[output_type['new']]
    #             output_order_w.value = [output_order_w.options[0]]

            output_type_w.observe(update_output_order_w, 'value')

            def update_output_split1_w(output_order):
                global output_split1_w          
                output_split1_w.options = output_split1s[output_type_w.value][output_order['new']]
    #             output_split1_w.value = [output_split1s_w.options[0]]

            output_order_w.observe(update_output_split1_w, 'value')

            def update_output_split2_w(output_split1):
                global output_split2_w
                output_split2_w.options = output_split2s[output_type_w.value][output_order_w.value][output_split1['new']]
                output_split2_w.value = [output_split2_w.options[0]]

            output_split1_w.observe(update_output_split2_w, 'value')     

            ### plot data using initial selection
            plot_data_initial()

            ### observe selection and replot when anything changes
            output_type_w.observe(plot_data_output_type, 'value')
            output_order_w.observe(plot_data_output_type, 'value')
            output_split1_w.observe(plot_data_output_split1, 'value')
            output_split2_w.observe(plot_data_output_split2, 'value')

            
            ### box for displaying plot options
            plot_bottom_display_w = widgets.VBox([plot_html2_w, buttons_download_w])
            display(plot_bottom_display_w)
            
        
            
### create folder for results in user's storage space
temp_results_sim = 'results'
temp_results_plt = 'results'
permanent_results = "results"       
if not os.path.isdir(permanent_results):
    !{mkdir_os} {slash_os(permanent_results)}
if temp_results_sim != permanent_results:
    !{mkdir_os} {slash_os(temp_results_sim)}
if temp_results_plt != permanent_results:
    !{mkdir_os} {slash_os(temp_results_plt)}
temp_results_sim += '/'
temp_results_plt += '/'
permanent_results += '/'

# legend location
leg_loc = 'top-left'
        
# set up a dictionary of Output widgets
static_outputs = {i: widgets.Output() for i in range(0,3)}
dynamic_outputs = {i: widgets.Output() for i in range(0,3)}

output_vbox = static_outputs.copy()
output_vbox[0] = widgets.VBox([static_outputs[0], dynamic_outputs[0]])
output_vbox[1] = widgets.VBox([static_outputs[1], dynamic_outputs[1]])
output_vbox[2] = widgets.VBox([static_outputs[2], dynamic_outputs[2]])

# add the Output widgets as tab childen
tab = widgets.Tab()
tab.children = list(output_vbox.values())

tab_labels = ['About', 'Run simulation', 'Plot data']
for i, title in static_outputs.items():
    tab.set_title(i, tab_labels[i])
    
with static_outputs[0]: tab_intro()

tab.observe(update_tab_display, names='selected_index')

display(tab)